In [1]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\esthe\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
df = (spark.read
              .option("sep", ";")
              .option("header", "false")
              .option("encoding", "UTF-8")
              .csv('../pyspark project/EMPRE.CSV')
              )

In [6]:
from pyspark.sql.functions import col, desc, count

### Inspecionando os dados do dataframe

In [7]:
#Conteúdo completo das primeiras 5 linhas do dataframe.
df.show(5, truncate=False)

+--------+--------------------------------------------------------+----+---+--------+---+----+
|_c0     |_c1                                                     |_c2 |_c3|_c4     |_c5|_c6 |
+--------+--------------------------------------------------------+----+---+--------+---+----+
|04631962|CLINICA OFTALMOLOGICA DR VANIA XIMENES LTDA             |2062|49 |0,00    |05 |null|
|04631963|EXPRESSO COSTA LTDA                                     |2062|49 |0,00    |01 |null|
|04631964|PAULO BRAGA PROJETOS ESTRUTURAIS E CONSULTORIA S/S LTDA.|2232|49 |20000,00|01 |null|
|04631965|COTERMAQ CONSERTOS E TERCEIRIZACAO DE MAQUINAS LTDA     |2062|49 |0,00    |03 |null|
|04631967|ACACIA MEDICINA DO TRABALHO E SEGURANCA LTDA            |2240|49 |10000,00|03 |null|
+--------+--------------------------------------------------------+----+---+--------+---+----+
only showing top 5 rows



In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



### Limpando e alterando o dataframe

In [9]:
#Renomeando as colunas (headers) e substituindo valores "null" por vazio.
df2 = df.withColumnRenamed("_c0","cnpj_basico") \
        .withColumnRenamed("_c1","razao_social") \
        .withColumnRenamed("_c2","cod_natureza_juridica") \
        .withColumnRenamed("_c3","qualificacao_responsavel") \
        .withColumnRenamed("_c4","capital_social")\
        .withColumnRenamed("_c5","cod_porte_empresa") \
        .withColumnRenamed("_c6","ente_fed_responsavel")

In [10]:
#Checando tipos de dados
df2.dtypes

[('cnpj_basico', 'string'),
 ('razao_social', 'string'),
 ('cod_natureza_juridica', 'string'),
 ('qualificacao_responsavel', 'string'),
 ('capital_social', 'string'),
 ('cod_porte_empresa', 'string'),
 ('ente_fed_responsavel', 'string')]

### Executando Queries 

#### A query abaixo registra na coluna "count" a quantidade de vezes em que os campos da coluna "cod_porte_empresa" se repetem, considerando que cada código representa:
<br> 00 – NÃO INFORMADO </br>
<br> 01 - MICRO EMPRESA </br>
<br> 03 - EMPRESA DE PEQUENO PORTE </br>
<br> 05 - DEMAIS </br>
#### Conclui-se que o maior número de registros é de micro empresas, com 2434195 registros.

In [11]:
df2.groupBy(col('cod_porte_empresa')).count().sort(desc("count"))

DataFrame[cod_porte_empresa: string, count: bigint]

#### As consultas abaixo calculam o valor máximo e o mínimo existentes na coluna referente ao capital social da empresa

In [12]:
max_value = df2.agg({"capital_social": "max"}).collect()[0]
print (max_value)

Row(max(capital_social)='9999999,00')


In [13]:
min_value = df2.agg({"capital_social": "min"}).collect()[0]
print (min_value)

Row(min(capital_social)='0,00')
